<a href="https://colab.research.google.com/github/Santiago-Quinteros/SQLAlchemy-Tutorial-With-Examples/blob/main/SQLAlchemy_Tutorial_With_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQLAlchemy Tutorial With Examples

A tutorial to remember SQL and SQLAlchemy

ChatGPT

https://www.datacamp.com/tutorial/sqlalchemy-tutorial-examples

https://sqlzoo.net/wiki/SQL_Tutorial



## What is SQLAlchemy?
SQLAlchemy is a Python library that helps you work with databases. It provides:

A Core layer for building raw SQL queries in Python.

An ORM (Object Relational Mapping) layer that lets you interact with database tables as if they were Python objects.

## Key Concepts
Database: A collection of data organized into tables.

Table: Similar to a spreadsheet, it contains rows and columns.

Row: A single record.

Column: A property of the record (e.g., name, age).

Engine: Connects to the database.

Session: Manages communication with the database.

## Key Points About Queries:

Queries are instructions to interact with the database.

In SQLAlchemy, they are abstracted into Python objects and executed through a connection/session.

Common query types include SELECT (retrieve), INSERT (add), UPDATE (modify), and DELETE (remove).

In [36]:
pip install sqlalchemy

In [37]:
import sqlalchemy
sqlalchemy.__version__

'2.0.36'

In [38]:
import sqlalchemy as db
#Download and place in working folder
#https://www.kaggle.com/datasets/groleo/european-football-database?resource=download
engine = db.create_engine("sqlite:///european_database.sqlite")
conn = engine.connect()

What is an Engine?

The engine in SQLAlchemy is the core component responsible for connecting your Python application to a database. It's like the "bridge" between Python and the actual database.

Key Responsibilities of the Engine

* Establish Connection: The engine knows how to talk to your database (e.g., SQLite, PostgreSQL, MySQL).

* Execute Queries: It runs queries against the database.

* Manage Resources: It handles database connections efficiently.


In [43]:
# Use the inspector to get the list of tables
inspector = db.inspect(engine)

# Get the names of all tables in the database
table_names = inspector.get_table_names()

# Print the table names
print(table_names)

['divisions', 'matchs']


In [47]:
metadata = db.MetaData() #extracting the metadata (what is the data, type for example)
#metadata is an object that contains all the metadata from the tables
division= db.Table('divisions', metadata, autoload_with=engine) #Table object

In [44]:
print(repr(metadata.tables['divisions']))

Table('divisions', MetaData(), Column('division', TEXT(), table=<divisions>), Column('name', TEXT(), table=<divisions>), Column('country', TEXT(), table=<divisions>), schema=None)


In [45]:
print(division.columns.keys())

['division', 'name', 'country']


In [48]:
query=division.select() #Make a select query
print(query)

SELECT divisions.division, divisions.name, divisions.country 
FROM divisions


In [49]:
# the queries are executed from the connection

exe = conn.execute(query) #executing the query
result = exe.fetchmany(5) #extracting top 5 results
print(result)

[('B1', 'Division 1A', 'Belgium'), ('D1', 'Bundesliga', 'Deutschland'), ('D2', '2. Bundesliga', 'Deutschland'), ('E0', 'Premier League', 'England'), ('E1', 'EFL Championship', 'England')]


# Creating Tables

First, we will create a new database called datacamp.sqlite. The create_engine will create a new database automatically if there is no database with the same name. So, creating and connecting are pretty much similar.

After that, we will connect the database and create a metadata object.

We will use SQLAlchmys Table function to create a table called “Student”

It consists of columns:

Id: Integer and primary key

Name: String and non-nullable

Major: String and default = “Math”

Pass: Boolean and default =True

We have created the structure of the table. Lets add it to the database using `metadata.create_all(engine)`.